In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from networkx.algorithms import community
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')

## 1. Data Loading
Load cubic spline interpolated price data for all 4 metals (6 series each).

In [ ]:
# Load data files
metals = {
    'Cobalt': pd.read_csv('data/ALL_cobalt_prices_cubic_spline.csv', parse_dates=['Date'], index_col='Date'),
    'Copper': pd.read_csv('data/ALL_copper_prices_cubic_spline.csv', parse_dates=['Date'], index_col='Date'),
    'Lithium': pd.read_csv('data/ALL_lithium_prices_cubic_spline.csv', parse_dates=['Date'], index_col='Date'),
    'Nickel': pd.read_csv('data/ALL_nickel_prices_cubic_spline.csv', parse_dates=['Date'], index_col='Date')
}

# Display basic info
for metal, df in metals.items():
    print(f"{metal}: {df.shape[0]} rows, {df.shape[1]} series")
    print(f"  Date range: {df.index.min()} to {df.index.max()}")
    print(f"  Columns: {', '.join(df.columns)}")
    print()

## 2. Compute Returns

Calculate regular returns (percent change) for all price series.

In [ ]:
# Convert prices to returns for all metals
metals_returns = {}

for metal, df in metals.items():
    # Calculate returns: (price_t - price_t-1) / price_t-1
    returns = df.pct_change()
    metals_returns[metal] = returns
    
    print(f"{metal} returns:")
    print(f"  Shape: {returns.shape}")
    print(f"  Mean return range: {returns.mean().min():.6f} to {returns.mean().max():.6f}")
    print(f"  Std dev range: {returns.std().min():.4f} to {returns.std().max():.4f}")
    print()

# Update the metals dictionary to use returns
metals = metals_returns

## 3. Common Timeframe Identification

Find the overlapping period where all 24 series have valid data.

In [ ]:
# Combine all dataframes
all_series = pd.concat([df.add_prefix(f'{metal}_') for metal, df in metals.items()], axis=1)

# Exclude LISAME series
lisame_cols = [col for col in all_series.columns if 'LISAME' in col]
if lisame_cols:
    print(f"Excluding LISAME series: {lisame_cols}")
    all_series = all_series.drop(columns=lisame_cols)

print(f"\nCombined dataset: {all_series.shape}")
print(f"Date range before alignment: {all_series.index.min()} to {all_series.index.max()}")
print(f"\nMissing values per series:")
print(all_series.isnull().sum())

In [ ]:
# Find common timeframe with complete data
# Strategy: Find the period where all 24 series have non-null values

# Count non-null values per date across all series
complete_dates = all_series.notna().all(axis=1)
complete_data = all_series[complete_dates]

if len(complete_data) == 0:
    # Fallback: use the intersection of date ranges with most data
    print("\nNo dates with all 24 series complete. Finding best overlap...")
    
    # For each series, find first and last valid dates
    series_ranges = {}
    for col in all_series.columns:
        valid_data = all_series[col].dropna()
        if len(valid_data) > 0:
            series_ranges[col] = (valid_data.index.min(), valid_data.index.max())
    
    # Find the common period (latest start, earliest end)
    common_start = max(start for start, end in series_ranges.values())
    common_end = min(end for start, end in series_ranges.values())
    
    print(f"\nCommon period: {common_start} to {common_end}")
    
    # Extract this period
    aligned_data = all_series.loc[common_start:common_end].copy()
    
    # Check coverage
    coverage = aligned_data.notna().sum() / len(aligned_data) * 100
    print(f"\nData coverage in common period:")
    print(coverage.sort_values(ascending=False))
    
    # Use only series with >90% coverage
    good_series = coverage[coverage > 90].index.tolist()
    if len(good_series) < 24:
        print(f"\nUsing {len(good_series)} series with >90% coverage")
        aligned_data = aligned_data[good_series]
    
    # Forward fill small gaps (max 5 days)
    aligned_data = aligned_data.fillna(method='ffill', limit=5)
    
    # Drop any remaining rows with NaN
    aligned_data = aligned_data.dropna()
else:
    common_start = complete_data.index.min()
    common_end = complete_data.index.max()
    aligned_data = complete_data

# Exclude weekend days (Saturday=5, Sunday=6)
weekdays_only = aligned_data.index.dayofweek < 5
aligned_data = aligned_data[weekdays_only]

print(f"\n✓ Final aligned dataset (weekdays only):")
print(f"  Period: {aligned_data.index.min()} to {aligned_data.index.max()}")
print(f"  Duration: {(aligned_data.index.max() - aligned_data.index.min()).days} days")
print(f"  Observations: {len(aligned_data)} (weekdays only)")
print(f"  Series: {len(aligned_data.columns)}")

## 4. Correlation Analysis

Compute pairwise correlations across all aligned return series.

In [ ]:
# Compute correlation matrix
corr_matrix = aligned_data.corr()

print(f"Correlation matrix: {corr_matrix.shape}")
print(f"\nCorrelation statistics:")
print(f"  Mean: {corr_matrix.values[np.triu_indices_from(corr_matrix.values, k=1)].mean():.3f}")
print(f"  Median: {np.median(corr_matrix.values[np.triu_indices_from(corr_matrix.values, k=1)]):.3f}")
print(f"  Min: {corr_matrix.values[np.triu_indices_from(corr_matrix.values, k=1)].min():.3f}")
print(f"  Max: {corr_matrix.values[np.triu_indices_from(corr_matrix.values, k=1)].max():.3f}")

In [ ]:
# Visualize full correlation matrix
plt.figure(figsize=(16, 14))
sns.heatmap(corr_matrix, cmap='RdYlGn', center=0, vmin=-1, vmax=1,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix: All Metal Return Series', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

## 5. Network Visualization

Create a correlation network graph to reveal relationships between metals and series.

**Design choices:**
- **Nodes**: Each return series
- **Node colors**: Color-coded by metal (Cobalt=blue, Copper=orange, Lithium=green, Nickel=purple)
- **Edges**: Shown only for correlations > 0.3 to reduce clutter
- **Edge thickness**: Proportional to correlation strength
- **Layout**: Force-directed layout with community detection to reveal clustering patterns

In [ ]:
# Create network graph
G = nx.Graph()

# Add nodes with metadata
metal_colors = {
    'Cobalt': '#3498db',   # Blue
    'Copper': '#e67e22',   # Orange
    'Lithium': '#27ae60',  # Green
    'Nickel': '#9b59b6'    # Purple
}

node_colors = []
node_labels = {}

for col in aligned_data.columns:
    # Determine metal from column name
    metal = col.split('_')[0]
    G.add_node(col, metal=metal)
    node_colors.append(metal_colors[metal])
    node_labels[col] = col

# Add edges for significant correlations (threshold: 0.3)
correlation_threshold = 0.3
edge_weights = []

for i, col1 in enumerate(aligned_data.columns):
    for col2 in aligned_data.columns[i+1:]:
        corr = corr_matrix.loc[col1, col2]
        if abs(corr) > correlation_threshold:
            G.add_edge(col1, col2, weight=abs(corr))
            edge_weights.append(abs(corr))

print(f"Network statistics:")
print(f"  Nodes: {G.number_of_nodes()}")
print(f"  Edges: {G.number_of_edges()} (correlations > {correlation_threshold})")
print(f"  Average degree: {sum(dict(G.degree()).values()) / G.number_of_nodes():.2f}")

In [ ]:
# Detect communities using Louvain method
communities = community.greedy_modularity_communities(G, weight='weight')

print(f"\nCommunity detection:")
print(f"  Number of communities: {len(communities)}")
for i, comm in enumerate(communities):
    metals_in_comm = {}
    for node in comm:
        metal = node.split('_')[0]
        metals_in_comm[metal] = metals_in_comm.get(metal, 0) + 1
    print(f"  Community {i+1}: {len(comm)} nodes - {dict(metals_in_comm)}")

In [ ]:
# Create network visualization
fig, ax = plt.subplots(figsize=(20, 20))

# Use spring layout with community structure
pos = nx.spring_layout(G, k=2, iterations=50, seed=42, weight='weight')

# Draw edges with varying thickness
edge_widths = [G[u][v]['weight'] * 3 for u, v in G.edges()]
nx.draw_networkx_edges(G, pos, width=edge_widths, alpha=0.3, edge_color='gray')

# Draw nodes colored by metal
nx.draw_networkx_nodes(G, pos, node_color=node_colors, 
                       node_size=800, alpha=0.9, linewidths=2, edgecolors='white')

# Add legend for metals (no node labels to avoid clutter)
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', 
                              markerfacecolor=color, markersize=15, label=metal)
                   for metal, color in metal_colors.items()]
ax.legend(handles=legend_elements, loc='upper right', fontsize=14, framealpha=0.9)

plt.title('Cross-Metal Correlation Network\n(Edge thickness ∝ correlation strength, correlation > 0.3 shown)',
          fontsize=16, pad=20)
plt.axis('off')
plt.tight_layout()
plt.show()

## 6. Within-Metal vs Cross-Metal Correlations

Analyze correlation patterns within and between metals (based on returns).

In [ ]:
# Separate within-metal and cross-metal correlations
within_metal_corrs = []
cross_metal_corrs = []

for i, col1 in enumerate(aligned_data.columns):
    metal1 = col1.split('_')[0]
    for col2 in aligned_data.columns[i+1:]:
        metal2 = col2.split('_')[0]
        corr = corr_matrix.loc[col1, col2]
        
        if metal1 == metal2:
            within_metal_corrs.append((metal1, corr))
        else:
            cross_metal_corrs.append((f"{metal1}-{metal2}", corr))

print("Within-metal correlations:")
print(f"  Count: {len(within_metal_corrs)}")
print(f"  Mean: {np.mean([c for _, c in within_metal_corrs]):.3f}")
print(f"  Median: {np.median([c for _, c in within_metal_corrs]):.3f}")

print("\nCross-metal correlations:")
print(f"  Count: {len(cross_metal_corrs)}")
print(f"  Mean: {np.mean([c for _, c in cross_metal_corrs]):.3f}")
print(f"  Median: {np.median([c for _, c in cross_metal_corrs]):.3f}")

In [ ]:
# Visualize distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Within-metal
for metal in metals.keys():
    metal_corrs = [c for m, c in within_metal_corrs if m == metal]
    if metal_corrs:
        axes[0].hist(metal_corrs, bins=20, alpha=0.6, label=metal, color=metal_colors[metal])

axes[0].set_xlabel('Correlation', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Within-Metal Correlations', fontsize=14)
axes[0].legend()
axes[0].axvline(np.mean([c for _, c in within_metal_corrs]), color='red', 
                linestyle='--', linewidth=2, label=f'Mean: {np.mean([c for _, c in within_metal_corrs]):.2f}')
axes[0].grid(alpha=0.3)

# Cross-metal
axes[1].hist([c for _, c in cross_metal_corrs], bins=30, alpha=0.7, color='steelblue')
axes[1].set_xlabel('Correlation', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Cross-Metal Correlations', fontsize=14)
axes[1].axvline(np.mean([c for _, c in cross_metal_corrs]), color='red', 
                linestyle='--', linewidth=2, label=f'Mean: {np.mean([c for _, c in cross_metal_corrs]):.2f}')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Key Insights

**Timeframe:** The analysis uses the common period where all series have sufficient data coverage.

**Correlation Patterns:**
- Within-metal correlations tend to be higher (series from the same metal move together)
- Cross-metal correlations reveal economic relationships between different commodity markets
- The network graph reveals natural clustering, with some cross-metal connections indicating shared market drivers

**Network Structure:**
- Node colors clearly distinguish the 4 metals
- Edge thickness shows correlation strength
- Community detection may reveal groups of series that move together regardless of metal type
- Isolated nodes or clusters suggest series with unique price dynamics